In [6]:
import pandas as pd

base = pd.DataFrame()
for i in range(103,112):
    print(i)
    temp = pd.read_csv(f"{i}_base0.csv",thousands=",")
    temp['year'] = f"{i}"
    base = pd.concat([base, temp], ignore_index=True)

base.info()
base['year'].unique()

103
104
105
106
107
108
109
110
111
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4724 entries, 0 to 4723
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   學校代碼      4724 non-null   object 
 1   學校名稱      4724 non-null   object 
 2   專任教師數男    4724 non-null   int64  
 3   專任教師數女    4724 non-null   int64  
 4   兼任教師數男    4724 non-null   int64  
 5   兼任教師數女    4724 non-null   int64  
 6   職員數男      4724 non-null   int64  
 7   職員數女      4724 non-null   int64  
 8   班級數       4724 non-null   int64  
 9   學生數男      4724 non-null   int64  
 10  學生數女      4724 non-null   int64  
 11  上學年畢業生數男  4724 non-null   int64  
 12  上學年畢業生數女  4724 non-null   int64  
 13  year      4724 non-null   object 
 14  縣市代碼      2115 non-null   float64
 15  縣市名稱      2115 non-null   object 
dtypes: float64(1), int64(11), object(4)
memory usage: 590.6+ KB


array(['103', '104', '105', '106', '107', '108', '109', '110', '111'],
      dtype=object)

In [ ]:
base109['teach_fulltime'] = base109['專任教師數男'] + base109['專任教師數女']
base109['teach_parttime'] = base109['兼任教師數男'] + base109['兼任教師數女']
base109['teach_all'] = base109['teach_fulltime'] + base109['teach_parttime']
base109['emp'] = base109['職員數男'] + base109['職員數女']
base109['stud'] = base109['學生數男'] + base109['學生數女']
base109['grad'] = base109['上學年畢業生數男'] + base109['上學年畢業生數女']
print(base109.tail())
print(base109.info())